# ML Olympiad for Students - TopVistos EUA

### Contexto
As comunidades empresariais nos Estados Unidos enfrentam uma alta demanda por recursos humanos, mas um dos desafios constantes é identificar e atrair o talento certo, que é talvez o elemento mais importante para se manter competitivo. Empresas nos Estados Unidos procuram indivíduos trabalhadores, talentosos e qualificados tanto localmente quanto no exterior.



A Lei de Imigração e Nacionalidade (INA) dos EUA permite que trabalhadores estrangeiros venham trabalhar nos Estados Unidos temporária ou permanentemente. A lei também protege os trabalhadores americanos contra impactos adversos em seus salários ou condições de trabalho, garantindo que os empregadores americanos cumpram os requisitos legais ao contratar trabalhadores estrangeiros para suprir a escassez de mão de obra. Os programas de imigração são administrados pelo Escritório de Certificação de Trabalho Estrangeiro (OFLC).



O OFLC processa pedidos de certificação de emprego para empregadores que buscam trazer trabalhadores estrangeiros para os Estados Unidos e concede certificações nos casos em que os empregadores podem demonstrar que não há trabalhadores americanos suficientes disponíveis para realizar o trabalho com salários que atendam ou excedam o salário pago para a ocupação na área de emprego pretendida.



### Objetivo
No ano fiscal de 2016, o OFLC processou 775.979 pedidos de empregadores para 1.699.957 posições de certificações de trabalho temporárias e permanentes. Isso representou um aumento de nove por cento no número total de pedidos processados em relação ao ano anterior. O processo de revisar cada caso está se tornando uma tarefa tediosa à medida que o número de candidatos aumenta a cada ano.



O aumento do número de candidatos a cada ano demanda uma solução baseada em Aprendizado de Máquina que possa ajudar na pré-seleção dos candidatos com maiores chances de aprovação de VISTO. O OFLC contratou sua empresa, TopVistos, para soluções baseadas em dados. Como Cientista de Dados, você deve analisar os dados fornecidos e, com a ajuda de um modelo de classificação:



Facilitar o processo de aprovação de vistos.



Recomendar um perfil adequado para os candidatos para os quais o visto deve ser certificado ou negado, com base nos fatores que influenciam significativamente o status do caso.

### Dataset Description

Os dados contêm diferentes atributos do empregado e do empregador. O dicionário de dados detalhado é fornecido abaixo.


Temos alguns arquivos disponibilizados para o treino dos modelos. Note que para seu modelo ser corretamente avaliado, você precisa enviar os seus dados no formato do arquivo sample_submission.csv.


#### Arquivos
- **train.csv** - dataset de treino
- **test.csv** - dataset de testes
- **sample_submission.csv** - um exemplo de como o arquivo enviado deve ser composto

#### Dicionário dos Dados

- **id_do_caso:** ID de cada solicitação de visto
- **continente:** Informação do continente do empregado
- **educação_do_empregado:** Informação sobre a educação do empregado
- **tem_experiência_de_trabalho:** O empregado possui alguma experiência de trabalho? S = Sim; N = Não
- **requer_treinamento_de_trabalho:** O empregado requer algum treinamento de trabalho? S = Sim; N = Não
- **num_de_empregados:** Número de funcionários na empresa do empregador
- **ano_de_estabelecimento:** Ano em que a empresa do empregador foi estabelecida
- **região_de_emprego:** Informação da região de emprego pretendida pelo trabalhador estrangeiro nos EUA.
- **salário_prevalecente:** Salário médio pago a trabalhadores em ocupações semelhantes na área de emprego pretendida. O objetivo do salário prevalecente é garantir que o trabalhador estrangeiro não seja subremunerado em comparação com outros trabalhadores que oferecem o mesmo serviço ou serviço similar na mesma área de emprego.
- **unidade_de_salário:** Unidade do salário prevalecente. Os valores incluem Por Hora, Por Semana, Por Mês e Por Ano.
- **posição_em_tempo_integral:** A posição de trabalho é em tempo integral? S = Posição em Tempo Integral; N = Posição em Meio Período
- **status_do_caso:** Indicador se o visto foi certificado ou negado


In [ ]:
import pandas as pd
pd.set_option("display.max_columns",100)
import numpy as np

         # Clsassification Models
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeRegressor

          # Modelling
from sklearn.model_selection import train_test_split

          # Testing
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report



from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV



import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.shape

(17836, 12)

-------------

status_do_caso: Indicador do facto de o visto ter sido aprovado ou não. Criei-o como 'Negado':0, 'Aprovado':1.

In [ ]:
d = {'Negado':0, 'Aprovado':1}
df["status_do_caso"]=df["status_do_caso"].map(d)
df["status_do_caso"].unique()

array([0, 1])

-----------------

In [ ]:
df_test = pd.read_csv("test.csv")
df = df.append(df_test)

In [ ]:
df.shape, df_test.shape

((25480, 12), (7644, 11))

## Exploratory Data Analysis

É o processo de compreensão, exploração e visualização de dados. Neste processo, iremos limpar os dados, caracterizar e efectuar análises estatísticas, visualizar os dados e interpretar os resultados.

In [ ]:
df.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,EZYV10567,Europa,Ensino Médio,N,S,2087,1855,Sul,69711.24,Ano,S,0.0
1,EZYV5505,Ásia,Mestrado,S,N,5991,2003,Meio-Oeste,52931.38,Ano,S,1.0
2,EZYV5207,Ásia,Ensino Médio,N,N,1426,2000,Ilha,110830.21,Ano,S,0.0
3,EZYV7545,Ásia,Ensino Médio,N,N,3846,1992,Meio-Oeste,91884.68,Semana,S,0.0
4,EZYV16071,Ásia,Ensino Superior,S,N,3957,1949,Nordeste,138155.24,Ano,S,1.0


In [ ]:
df.isnull().sum()

id_do_caso                           0
continente                           0
educacao_do_empregado                0
tem_experiencia_de_trabalho          0
requer_treinamento_de_trabalho       0
num_de_empregados                    0
ano_de_estabelecimento               0
regiao_de_emprego                    0
salario_prevalecente                 0
unidade_de_salario                   0
posicao_em_tempo_integral            0
status_do_caso                    7644
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25480 entries, 0 to 7643
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_do_caso                      25480 non-null  object 
 1   continente                      25480 non-null  object 
 2   educacao_do_empregado           25480 non-null  object 
 3   tem_experiencia_de_trabalho     25480 non-null  object 
 4   requer_treinamento_de_trabalho  25480 non-null  object 
 5   num_de_empregados               25480 non-null  int64  
 6   ano_de_estabelecimento          25480 non-null  int64  
 7   regiao_de_emprego               25480 non-null  object 
 8   salario_prevalecente            25480 non-null  float64
 9   unidade_de_salario              25480 non-null  object 
 10  posicao_em_tempo_integral       25480 non-null  object 
 11  status_do_caso                  17836 non-null  float64
dtypes: float64(2), int64(2), object(8

## Feature Engineering

É o processo de criação de novas características utilizando as características existentes no conjunto de dados. Neste processo, as características existentes são manipuladas ou combinadas para tornar os dados mais significativos, melhorar o desempenho do modelo e obter melhores resultados.

---------------

Neste caso, atribuí um valor de 1 para o ensino secundário, 3 para o mestrado, 2 para o ensino superior e 4 para o doutoramento, porque é mais lógico que todos os Estados aceitem pessoas com formação superior.

In [ ]:
d = {'Ensino Médio':1, 'Mestrado':3, 'Ensino Superior':2, 'Doutorado':4}
df["educacao_do_empregado"]=df["educacao_do_empregado"].map(d)
df["educacao_do_empregado"].unique()

array([1, 3, 2, 4])

----------

Se o trabalhador tem experiência profissional. S = Sim, ou seja, 1; N = Não, ou seja, 0.

In [ ]:
d = {"S":1,"N":0}
df["tem_experiencia_de_trabalho"]=df["tem_experiencia_de_trabalho"].map(d)
df["tem_experiencia_de_trabalho"].unique()

array([0, 1])

------------

Se o trabalhador necessita de formação profissional. S = Sim, ou seja, 0; N = Não, ou seja, 1. Porque, se precisar de formação, está em desvantagem em relação aos que não precisam.

In [ ]:
d={"S":1,"N":0}
df["requer_treinamento_de_trabalho"]=df["requer_treinamento_de_trabalho"].map(d)
df["requer_treinamento_de_trabalho"].unique()

array([1, 0])

-------

num_of_employees e year_of_establishment: Utilizando estas duas propriedades, pode criar uma nova propriedade chamada "average_annual_employee_growth". Esta propriedade mostra a rapidez com que a empresa tem crescido ao longo dos anos.

In [ ]:
df['ano_de_estabelecimento'] = 2023 - df['ano_de_estabelecimento']

---------------

Calculei o salário anual utilizando as colunas "salário_prevalecente" (salário actual) e "unidade_de_salário" (unidade salarial).

Em primeiro lugar, é necessário verificar qual é a unidade de salário. Se a unidade de salário for horária, semanal ou mensal, temos de a converter em salário anual. Por exemplo, podemos normalmente multiplicar o salário por hora por 2080 (40 horas/semana * 52 semanas/ano) para converter o salário por hora em salário anual, multiplicar o salário semanal por 52 (52 semanas/ano) para converter o salário semanal em salário anual e multiplicar o salário mensal por 12 (12 meses/ano) para converter o salário mensal em salário anual. Se a unidade salarial já for anual, não é necessário alterar o salário.

In [ ]:
df.loc[df['unidade_de_salario'] == 'Hora', 'salario_prevalecente'] *= 2080
df.loc[df['unidade_de_salario'] == 'Semana', 'salario_prevalecente'] *= 52
df.loc[df['unidade_de_salario'] == 'Mes', 'salario_prevalecente'] *= 12
df['salario_anual'] = df['salario_prevalecente']

-------

Analisei o impacto do trabalho a tempo inteiro ou a tempo parcial no salário combinando o atributo "Impact of Employment Status on Salary", ou seja, o atributo "posição_em_tempo_integral" e o atributo "salário_anual".

In [ ]:
df['posicao_em_tempo_integral'] = df['posicao_em_tempo_integral'].map({'S': 1, 'N': 0})
df['efeito_salario_emprego'] = df['salario_anual'] * df['posicao_em_tempo_integral']

-------------

educação_do_empregado e tem_experiência_de_trabalho: Se estas colunas contiverem dados categóricos, combinei estas duas propriedades para criar uma nova propriedade chamada "mão-de-obra qualificada".

In [ ]:
df['qualificado_trabalho'] = df['educacao_do_empregado'] + df['tem_experiencia_de_trabalho']

-------------

------------

empresa_grande: num_de_empregados özelliğini kullanarak, firmanın büyük bir firma olup olmadığını belirleyebiliriz. Genellikle daha büyük firmaların vize başvuruları daha olasıdır.

In [ ]:
df['empresa_grande'] = df['num_de_empregados'].apply(lambda x: 1 if x > 500 else 0)

-------------

salario_em_relacao_ao_num_de_empregados: dividindo a propriedade salario_prevalecente pela propriedade num_de_empregados, obtém-se o salário médio de cada empregado. Isto dá uma ideia da política salarial global da empresa.

In [ ]:
df['salario_em_relacao_ao_num_de_empregados'] = df['salario_prevalecente'] / df['num_de_empregados']

----------------

requer_treinamento_de_trabalho e tem_experiencia_de_trabalho: Ao combinar estas duas características, ajuda a compreender melhor a capacidade de um empregado para fazer o trabalho. Se um empregado tem experiência de trabalho mas ainda precisa de formação no local de trabalho, isso talvez indique que o empregado não tem um conjunto de competências específicas.

In [ ]:
df['trabalho_e_experiencia'] = df['requer_treinamento_de_trabalho'] + df['tem_experiencia_de_trabalho']

--------------

salario_abaixo_media: Esta propriedade determina se o salário do candidato a emprego é inferior ao salário médio de outros empregados em empregos semelhantes.

In [ ]:
media_salario = df['salario_prevalecente'].mean()
df['salario_abaixo_media'] = df['salario_prevalecente'].apply(lambda x: 1 if x < media_salario else 0)

--------

In [ ]:
df

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso,salario_anual,efeito_salario_emprego,qualificado_trabalho,empresa_grande,salario_em_relacao_ao_num_de_empregados,trabalho_e_experiencia,salario_abaixo_media
0,EZYV10567,Europa,1,0,1,2087,168,Sul,69711.24,Ano,1,0.0,69711.24,69711.24,1,1,33.402607,1,1
1,EZYV5505,Ásia,3,1,0,5991,20,Meio-Oeste,52931.38,Ano,1,1.0,52931.38,52931.38,4,1,8.835149,1,1
2,EZYV5207,Ásia,1,0,0,1426,23,Ilha,110830.21,Ano,1,0.0,110830.21,110830.21,1,1,77.721045,0,1
3,EZYV7545,Ásia,1,0,0,3846,31,Meio-Oeste,4778003.36,Semana,1,0.0,4778003.36,4778003.36,1,1,1242.330567,0,0
4,EZYV16071,Ásia,2,1,0,3957,74,Nordeste,138155.24,Ano,1,1.0,138155.24,138155.24,3,1,34.914137,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7639,EZYV19907,América do Norte,2,0,0,66,15,Oeste,146845.92,Hora,1,NaN,146845.92,146845.92,2,0,2224.938182,0,1
7640,EZYV576,Ásia,1,1,0,4040,29,Meio-Oeste,60158.59,Ano,1,NaN,60158.59,60158.59,2,1,14.890740,1,1
7641,EZYV1804,Ásia,3,1,0,32503,98,Sul,44555.12,Ano,1,NaN,44555.12,44555.12,4,1,1.370800,1,1
7642,EZYV13543,América do Norte,3,1,0,1894,23,Sul,97830.73,Ano,1,NaN,97830.73,97830.73,4,1,51.652973,1,1


In [ ]:
del df["id_do_caso"]
data = pd.get_dummies(df,drop_first=True)

-------

In [ ]:
train = data[:17836]
test = data[17836:]

In [ ]:
x = train.drop("status_do_caso",axis=1)
y = train["status_do_caso"]

In [ ]:
sm = SMOTE(random_state=42)
x_res, y_res = sm.fit_resample(x, y)
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2, random_state=42)

# Parametre ızgarası
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 4, 5],
    "subsample": [0.5, 0.7, 1.0],
}

In [ ]:
# Grid Search
grid_search = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,  # 3-fold cross validation
    scoring='accuracy',  # doğruluk skorunu kullan
    verbose=2,  # sonuçların yazdırılması
    n_jobs=-1,  # tüm çekirdekleri kullan
)
grid_result = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


In [ ]:

# En iyi parametrelerle modeli eğit
best_params = grid_result.best_params_
model = GradientBoostingClassifier(**best_params)
model.fit(x_train, y_train)
print(f"En iyi hiperparametreler: {grid_search.best_params_}")
print(f"En yüksek doğruluk skoru: {grid_search.best_score_:.4f}")

En iyi hiperparametreler: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.7}
En yüksek doğruluk skoru: 0.7951


In [ ]:
# Tahmin
guess = model.predict(test.drop("status_do_caso", axis=1))

In [ ]:
# Sonuçları kaydet
result = pd.DataFrame()
result["id_do_caso"] =df_test["id_do_caso"]
result["status_do_caso"] = guess
result["status_do_caso"] = result["status_do_caso"].astype('int32')
result.to_csv("sample_submission_egitim_sayilari_artti.csv",index=False)